<a href="https://colab.research.google.com/github/dotsnangles/image_detection_mini_prjt/blob/master/taco_%EB%AF%B8%EB%8B%88_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5_%ED%8C%8C%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 원본 데이터 세트 로드

In [ ]:
!gdown --id 1aDE9nMZQ4_Kkfv2HRa7HGyRU-jbxhc7g

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1aDE9nMZQ4_Kkfv2HRa7HGyRU-jbxhc7g
To: /content/taco-trash-dataset.zip
100% 2.99G/2.99G [00:23<00:00, 127MB/s]


In [ ]:
!unzip /content/taco-trash-dataset.zip

Archive:  /content/taco-trash-dataset.zip
  inflating: best-checkpoint-003epoch.bin  
  inflating: data/annotations.json   
  inflating: data/batch_1/000000.jpg  
  inflating: data/batch_1/000001.jpg  
  inflating: data/batch_1/000003.jpg  
  inflating: data/batch_1/000004.jpg  
  inflating: data/batch_1/000005.jpg  
  inflating: data/batch_1/000006.jpg  
  inflating: data/batch_1/000007.jpg  
  inflating: data/batch_1/000008.jpg  
  inflating: data/batch_1/000010.jpg  
  inflating: data/batch_1/000011.jpg  
  inflating: data/batch_1/000012.jpg  
  inflating: data/batch_1/000013.jpg  
  inflating: data/batch_1/000014.jpg  
  inflating: data/batch_1/000015.jpg  
  inflating: data/batch_1/000016.jpg  
  inflating: data/batch_1/000017.jpg  
  inflating: data/batch_1/000019.jpg  
  inflating: data/batch_1/000021.jpg  
  inflating: data/batch_1/000022.jpg  
  inflating: data/batch_1/000023.jpg  
  inflating: data/batch_1/000024.jpg  
  inflating: data/batch_1/000025.jpg  
  inflating: data/

### yolov5 Git Clone

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 13442, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 13442 (delta 16), reused 11 (delta 7), pack-reused 13414
Receiving objects: 100% (13442/13442), 12.06 MiB | 41.43 MiB/s, done.
Resolving deltas: 100% (9376/9376), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.2 MB/s 


### 원본 데이터 세트 수정을 위한 COCO 데이터 세트 인터페이스 로드

In [ ]:
!pip install pycocotools

from pycocotools.coco import COCO
import numpy as np
import tqdm

### 어노테이션 데이터 로드

In [ ]:
annotation_path = '/content/data/annotations.json'

data_source = COCO(annotation_file=annotation_path)
catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
categories.sort(key=lambda x: x['id'])

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [ ]:
# super = [c['supercategory'] for c in categories]
# super = sorted(list(set(super)))
# classes = {}
# for i, s in enumerate(super):
#   classes[s] = i

### 기존 전이 학습에 사용된 데이터 세트에 맞춰 클래스 재분류

In [ ]:
names = [ 'Paper', 'Pet Bottle', 'Glass', 'Can', 'Vinly', 'Plastic','Junk','Styrofoam' ]  # class names
for i, name in enumerate(names):
    print(i, name)

0 Paper
1 Pet Bottle
2 Glass
3 Can
4 Vinly
5 Plastic
6 Junk
7 Styrofoam


In [ ]:
classes = {'Aluminium foil': 6,
            'Battery': 6,
            'Blister pack': 5,
            'Bottle': 2,
            'Bottle cap': 6,
            'Broken glass': 2,
            'Can': 3,
            'Carton': 0,
            'Cigarette': 6,
            'Cup': 6,
            'Food waste': 6,
            'Glass jar': 2,
            'Lid': 6,
            'Other plastic': 5,
            'Paper': 0,
            'Paper bag': 0,
            'Plastic bag & wrapper': 5,
            'Plastic container': 5,
            'Plastic glooves': 5,
            'Plastic utensils': 5,
            'Pop tab': 6,
            'Rope & strings': 6,
            'Scrap metal': 6,
            'Shoe': 6,
            'Squeezable tube': 5,
            'Straw': 5,
            'Styrofoam piece': 7,
            'Unlabeled litter': 6}

In [ ]:
labels_transfer = {}

for c in categories:
    labels_transfer[c['id']] = classes[c['supercategory']]

### 이미지/라벨 저장 폴더 생성

In [ ]:
!mkdir -p tmp/labels tmp/images
!ls

CONTRIBUTING.md  export.py   models	       setup.cfg  tutorial.ipynb
data		 hubconf.py  README.md	       tmp	  utils
detect.py	 LICENSE     requirements.txt  train.py   val.py


### yolov5 포맷에 맞춰 라벨 변환

In [ ]:
import shutil
import os

IMAGES_PATH = 'tmp/images/'
LABELS_PATH = 'tmp/labels/'

In [ ]:
img_ids = data_source.getImgIds()

for index, img_id in tqdm.tqdm(enumerate(img_ids)):
    img_info = data_source.loadImgs(img_id)[0]
    
    # img_dir: batch_x/.....jpg ---> batch_x_......jpg
    img_dir = img_info['file_name'].replace('/', '_')
    
    image_name = img_dir.split('.')[0]
    label_dir = LABELS_PATH + image_name + '.txt'
    
    height = img_info['height']
    width = img_info['width']

    # get images
    shutil.copy(f"/content/data/{img_info['file_name']}", os.path.join(IMAGES_PATH, img_dir))

    # get labels
    with open(label_dir, mode='w') as fp:
        annotation_id = data_source.getAnnIds(img_id)
        if len(annotation_id) == 0:
            fp.write('')
            continue
        boxes = np.zeros((0, 5))
        annotations = data_source.loadAnns(annotation_id)
        lines = ''
        for annotation in annotations:
            label = labels_transfer[annotation['category_id']]
            box = annotation['bbox']
            # some annotations have basically no width / height (extremely small), skip them
            if box[2] < 1 or box[3] < 1:
                continue
            # top_x,top_y,width,height ----> cen_x,cen_y,width,height
            # standardize to 0-1
            if width > height:
                halfpad = (width - height) / 2
                box[0] = round((box[0] + box[2] / 2) / width, 6)
                box[1] = round(((box[1] + box[3] / 2) + halfpad) / width, 6)
                box[2] = round(box[2] / width, 6)
                box[3] = round(box[3] / width, 6)
            else:
                halfpad = (height - width) / 2
                box[0] = round(((box[0] + box[2] / 2) + halfpad) / height, 6)
                box[1] = round((box[1] + box[3] / 2) / height, 6)
                box[2] = round(box[2] / height, 6)
                box[3] = round(box[3] / height, 6)
            # line: label x_center y_center width height
            lines = lines + str(label)
            for i in box:
                lines += ' ' + str(i)
            lines += '\n'
        fp.writelines(lines)

1500it [00:09, 164.56it/s]


In [ ]:
print(len(os.listdir(IMAGES_PATH)))
print(len(os.listdir(LABELS_PATH)))

1500
1500


### 이미지 리사이즈 및 패딩

In [ ]:
import cv2
from glob import glob

filenames = glob('/content/yolov5/tmp/images/*.*')

In [ ]:
for filename in filenames:
    img = cv2.imread(filename)

    # 이미지의 가로, 세로 중 큰 쪽을 640픽셀에 맞춰 리사이즈
    percent = 1
    if(img.shape[1] > img.shape[0]):
        percent = 640/img.shape[1]
    else:
        percent = 640/img.shape[0]

    img = cv2.resize(img, dsize=(0, 0), fx=percent, fy=percent, interpolation=cv2.INTER_LINEAR)
    
    # 어파인 변환을 활용한 블랙패딩
    y,x,h,w = (0,0,img.shape[0], img.shape[1])

    # 어파인 변환 시 이동값 설정
    w_x = (640-(w-x))/2  # [ 그림나머지/2 [그림] 그림나머지/2 ]
    h_y = (640-(h-y))/2

    if(w_x < 0):
        w_x = 0
    elif(h_y < 0):
        h_y = 0

    M = np.float32([[1,0,w_x], [0,1,h_y]])  # 어파인 변환 행렬
    img_re = cv2.warpAffine(img, M, (640, 640)) # 어파인 변환
    
    cv2.imwrite(filename, img_re)

In [ ]:
!zip -r dataset.zip tmp

In [ ]:
!pip install split-folders
import splitfolders

### 훈련/검증 세트 분할

In [ ]:
splitfolders.ratio('tmp', output='taco', seed=1337, ratio=(.8, 0.2,)) 

Copying files: 3000 files [00:00, 4960.68 files/s]


In [ ]:
!rm -rf /content/yolov5/taco

Check xem chia đúng không

In [ ]:
print(os.listdir('taco/train/images')[:5])
print(os.listdir('taco/train/labels')[:5])

['batch_5_000093.JPG', 'batch_12_000033.jpg', 'batch_1_000049.jpg', 'batch_11_000084.jpg', 'batch_8_000043.jpg']
['batch_10_000048.txt', 'batch_1_000027.txt', 'batch_8_000054.txt', 'batch_11_000077.txt', 'batch_14_000032.txt']


In [ ]:
print(len(os.listdir('/content/yolov5/taco/train/labels')))
print(len(os.listdir('/content/yolov5/taco/val/labels')))

1200
300


### 모니터링을 위한 wandb API 패키지 로드

In [ ]:
!pip install wandb
# !wandb disabled

     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 145 kB 87.6 MB/s 
     |████████████████████████████████| 181 kB 71.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=158d7751ff005b0e0f99b741a7b4997058dde857943b69b2656602c3484842f5
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
!ls

CONTRIBUTING.md  export.py   README.md	       train.py
data		 hubconf.py  requirements.txt  tutorial.ipynb
dataset.zip	 LICENSE     setup.cfg	       utils
detect.py	 models      tmp	       val.py


### 설정 파일 생성 및 수정

In [ ]:
!touch dataset.yaml

/content/yolov5/dataset.yaml 수정사항
```
train: /content/yolov5/taco/train  # train images (relative to 'path') 128 images
val: /content/yolov5/taco/val  # val images (relative to 'path') 128 images

# Classes
nc: 8  # number of classes
names: [ 'Paper', 'Pet Bottle', 'Glass', 'Can', 'Vinly', 'Plastic','Junk','Styrofoam' ]  # class names
```

/content/yolov5/models/yolov5m.yaml 수정사항

```
nc: 8  # number of classes
```

In [ ]:
# import gc
# gc.collect()

11

### 훈련 시작
[Weights & Biases 로그 링크](https://wandb.ai/dotsnangles/YOLOv5/runs/a0iv1zir?workspace=user-dotsnangles)

In [ ]:
# !python train.py --img 320 --batch 64 --epochs 1 --data /kaggle/input/yolov5tacosuper/taco-yolo_super.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt
# !python train.py --img 320 --batch 64 --epochs 50 --data /kaggle/input/yolov5tacosuper/taco-yolo_super.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt
!python train.py --img 640 --batch 32 --epochs 200 --data dataset.yaml --cfg models/yolov5m.yaml --weights yolov5m.pt
# !python train.py --img 320 --batch 64 --epochs 1 --data /kaggle/input/yolov5tacosuper/taco-yolo_super.yaml --cfg models/yolov5s.yaml --weights /kaggle/input/yolov5tacosuper/best.pt

wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin
train: weights=yolov5m.pt, cfg=models/yolov5m.yaml, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-196-g614ef11 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cl